# File I/O

So far we have discussed how to process data and how
to build, train, and test deep learning models.
However, at some point we will hopefully be happy enough
with the learned models that we will want
to save the results for later use in various contexts
(perhaps even to make predictions in deployment).
Additionally, when running a long training process,
the best practice is to periodically save intermediate results (checkpointing)
to ensure that we do not lose several days' worth of computation
if we trip over the power cord of our server.
Thus it is time to learn how to load and store
both individual weight vectors and entire models.
This section addresses both issues.


In [1]:
import flax
import jax
from flax import linen as nn
from flax.training import checkpoints
from jax import numpy as jnp
from d2l import jax as d2l

## (**Loading and Saving Tensors**)

For individual tensors, we can directly
invoke the `load` and `save` functions
to read and write them respectively.
Both functions require that we supply a name,
and `save` requires as input the variable to be saved.


In [2]:
x = jnp.arange(4)
jnp.save('x-file.npy', x)

We can now read the data from the stored file back into memory.


In [3]:
x2 = jnp.load('x-file.npy', allow_pickle=True)
x2

Array([0, 1, 2, 3], dtype=int32)

We can [**store a list of tensors and read them back into memory.**]


In [4]:
y = jnp.zeros(4)
jnp.save('xy-files.npy', [x, y])
x2, y2 = jnp.load('xy-files.npy', allow_pickle=True)
(x2, y2)

(Array([0., 1., 2., 3.], dtype=float32),
 Array([0., 0., 0., 0.], dtype=float32))

We can even [**write and read a dictionary that maps
from strings to tensors.**]
This is convenient when we want
to read or write all the weights in a model.


In [5]:
mydict = {'x': x, 'y': y}
jnp.save('mydict.npy', mydict)
mydict2 = jnp.load('mydict.npy', allow_pickle=True)
mydict2

array({'x': Array([0, 1, 2, 3], dtype=int32), 'y': Array([0., 0., 0., 0.], dtype=float32)},
      dtype=object)

## [**Loading and Saving Model Parameters**]

Saving individual weight vectors (or other tensors) is useful,
but it gets very tedious if we want to save
(and later load) an entire model.
After all, we might have hundreds of
parameter groups sprinkled throughout.
For this reason the deep learning framework provides built-in functionalities
to load and save entire networks.
An important detail to note is that this
saves model *parameters* and not the entire model.
For example, if we have a 3-layer MLP,
we need to specify the architecture separately.
The reason for this is that the models themselves can contain arbitrary code,
hence they cannot be serialized as naturally.
Thus, in order to reinstate a model, we need
to generate the architecture in code
and then load the parameters from disk.
(**Let's start with our familiar MLP.**)


In [6]:
class MLP(nn.Module):
    def setup(self):
        self.hidden = nn.Dense(256)
        self.output = nn.Dense(10)

    def __call__(self, x):
        return self.output(nn.relu(self.hidden(x)))

net = MLP()
X = jax.random.normal(jax.random.PRNGKey(d2l.get_seed()), (2, 20))
Y, params = net.init_with_output(jax.random.PRNGKey(d2l.get_seed()), X)

Next, we [**store the parameters of the model as a file**] with the name "mlp.params".


In [7]:
checkpoints.save_checkpoint('ckpt_dir', params, step=1, overwrite=True)

'ckpt_dir/checkpoint_1'

To recover the model, we instantiate a clone
of the original MLP model.
Instead of randomly initializing the model parameters,
we [**read the parameters stored in the file directly**].


In [8]:
clone = MLP()
cloned_params = flax.core.freeze(checkpoints.restore_checkpoint('ckpt_dir',
                                                                target=None))

Since both instances have the same model parameters,
the computational result of the same input `X` should be the same.
Let's verify this.


In [9]:
Y_clone = clone.apply(cloned_params, X)
Y_clone == Y

Array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]], dtype=bool)

## Summary

The `save` and `load` functions can be used to perform file I/O for tensor objects.
We can save and load the entire sets of parameters for a network via a parameter dictionary.
Saving the architecture has to be done in code rather than in parameters.

## Exercises

1. Even if there is no need to deploy trained models to a different device, what are the practical benefits of storing model parameters?
1. Assume that we want to reuse only parts of a network to be incorporated into a network having a different architecture. How would you go about using, say the first two layers from a previous network in a new network?
1. How would you go about saving the network architecture and parameters? What restrictions would you impose on the architecture?


[Discussions](https://discuss.d2l.ai/t/17994)
